## Ebola infected Macaque Sample Composition Trajectory Identification

```
Indices:

- c cell type
- g genes
- m samples
- k deformation polynomial degree
```

In [ ]:
import os
import pickle5
import numpy as np
import pandas as pd
#import matplotlib.pylab as plt
import matplotlib
import matplotlib.pyplot
from typing import Dict
from pyro.distributions.torch_distribution import TorchDistribution, TorchDistributionMixin
from torch.distributions.utils import probs_to_logits, logits_to_probs, broadcast_all, lazy_property
from torch.distributions import constraints
import torch
import pyro
from pyro.infer import SVI, Trace_ELBO
from typing import List, Dict
from boltons.cacheutils import cachedproperty
from pyro.distributions.torch_distribution import TorchDistribution, TorchDistributionMixin
from torch.distributions.utils import probs_to_logits, logits_to_probs, broadcast_all, lazy_property
from torch.distributions import constraints
from numbers import Number
import pyro.distributions as dist
import anndata
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
from sklearn.pipeline import make_pipeline

In [ ]:
from time_deconv.time_deconv import *

## Parameter Configuration

In [ ]:
device = torch.device("cuda:0")
dtype = torch.float32
dtype_np = np.float32

## Train Model

In [ ]:
bulk_anndata_path = "/home/nbarkas/disk2/deconvolution_method/datasets/ebov/load_data_python/ebov_bulk.h5ad"
sc_anndata_path = "/home/nbarkas/disk2/deconvolution_method/datasets/ebov/load_data_python/ebov_sc.h5ad"

In [ ]:
with open(bulk_anndata_path, 'rb') as fh:
    bulk_anndata  = anndata.read_h5ad(fh)
with open(sc_anndata_path, 'rb') as fh:
    sc_anndata = anndata.read_h5ad(fh)

In [ ]:
# select samples only after or on tp 0
bulk_anndata = bulk_anndata[bulk_anndata.obs['dpi_time'] >= 0,]

In [ ]:
ebov_dataset = DeconvolutionDataset(
    sc_anndata = sc_anndata,
    sc_celltype_col = "Subclustering_reduced",
    bulk_anndata = bulk_anndata,
    bulk_time_col = "dpi_time",
    dtype_np = dtype_np,
    dtype = dtype,
    device=device,
    feature_selection_method = 'overdispersed_bulk_and_high_sc' #'overdispersed_bulk'
)

In [ ]:
pseudo_time_reg_deconv = TimeRegularizedDeconvolution(
    dataset=ebov_dataset,
    polynomial_degree = 20,
    #basis_functions = "legendre",
    basis_functions = "polynomial",
    device=device,
    dtype=dtype)

In [ ]:
pseudo_time_reg_deconv.fit_model(n_iters=5_000, verbose=True, log_frequency=1000)

## Examine Outputs

In [ ]:
pseudo_time_reg_deconv.plot_loss()

In [ ]:
pseudo_time_reg_deconv.calculate_composition_trajectories(n_intervals = 1000)

In [ ]:
pseudo_time_reg_deconv.plot_composition_trajectories()

In [ ]:
pseudo_time_reg_deconv.plot_phi_g_distribution()

In [ ]:
pseudo_time_reg_deconv.plot_beta_g_distribution()
matplotlib.pyplot.yscale('log')

# Synthetic dataset generator

In [ ]:
import numpy as np

def sigmoid(x):
  
    z = np.exp(-x)
    sig = 1 / (1 + z)

    return sig


In [ ]:
batch_dict = generate_batch(pseudo_time_reg_deconv.dataset, device, dtype)
batch_dict.keys()

In [ ]:
def sample_sigmoid_proportions(num_cell_types, num_samples, t_m):
    # generate the celltype proportions
    effect_size = torch.rand(num_cell_types) # 0,1
    shift = torch.rand(num_cell_types)
    magnitude = torch.where(torch.rand(num_cell_types) < 0.5, -1., 1.)
    
    # Generate cell population mc
    cell_pop_cm = torch.zeros(num_cell_types, num_samples)
    for i in range(num_cell_types):
        cell_pop_cm[i,:] = torch.Tensor(list(sigmoid(magnitude[i]*x+shift[i]) for x in t_m)) * effect_size[i]
    cell_pop_cm = torch.nn.functional.softmax(cell_pop_cm, dim=0)
    
    return {
        
        'trajectory_params': {
            'type': 'sigmoid',
            'effect_size': effect_size,
            'shift': shift,
            'magnitude': magnitude,
            
        },
        'cell_pop_cm': cell_pop_cm
    }

In [ ]:
def simulate_with_sigmoid_proportions(start_time = -5, end_time = 5, step = 1, num_samples = 100,
                                     lib_size_mean = 1e6, lib_size_std = 2e5, use_betas = False):
    """Simulate bulk data with compositional changes"""
    
    # Discrete timepoints to sample from
    xs = torch.arange(start_time, end_time, step)
    
    # Number of celltypes are same as in main deconvolution
    num_cell_types = pseudo_time_reg_deconv.w_hat_gc.shape[1]
    
    # Sample the times for the samples
    t_m = xs[torch.randint(len(xs), (num_samples,))]
    
    proportions_sample = sample_sigmoid_proportions(num_cell_types, num_samples, t_m)
    
    cell_pop_cm = proportions_sample['cell_pop_cm']
    
    # Get phis and betas from main model
    # phi_g ~ 0.1 - 0.2
    
    phi_g = pyro.param("log_phi_posterior_loc_g").detach().exp().cpu()
    beta_g = pyro.param("log_beta_posterior_loc_g").detach().exp().cpu()
    
    # Get celltype profiles from the model
    w_hat_gc = pseudo_time_reg_deconv.w_hat_gc.detach().cpu()
    if use_betas:
        unnorm_w_hat_gc = w_hat_gc * beta_g[:,None]
    else:
        unnorm_w_hat_gc = w_hat_gc
    
    # Normalize
    w_gc = unnorm_w_hat_gc / unnorm_w_hat_gc.sum(0)
    
    # Get some library sizes 
    lib_sizes_m = torch.normal(
        mean=torch.full([num_samples], lib_size_mean), 
        std=torch.full([num_samples], lib_size_std)
    )
    
    # Get the NegBinomial means
    # consider: random component on w_gc?
    # b_gc -> gene + celltype specific distortion ( how does inference degrade as this increases )
    # sample b_gc from laplace (mu = 1, beta(scale) = )
    # Gamma(mean = 1, var = 1/rate)
    # rate = concentration = a
    # 1/a = var of gamma distribution
    # sample beta_cg 
    mu_mg = lib_sizes_m[:,None] * torch.matmul(cell_pop_cm.T, w_gc.transpose(-1, -2))
    
    # Sample a full matrix using phis from main model
    x_ng = NegativeBinomialAltParam(mu=mu_mg, phi = phi_g).sample()
    
    return {
        'cell_pop_cm': cell_pop_cm,
        't_m': t_m,
        'x_ng': x_ng,
        'trajectory_params': proportions_sample['trajectory_params'],
    }

In [ ]:
def plot_simulated_proportions(sim_res):
    """Plot simulated proportion results"""
    
    
    fig, ax = matplotlib.pyplot.subplots()
    o = torch.argsort(sim_res['t_m'])
    ax.plot(sim_res['t_m'][o], sim_res['cell_pop_cm'][:,o].T)
    ax.set_title('Simulated proportions')
    ax.set_xlabel('Set time')
    ax.set_ylabel('Proportions')
    
    return ax

In [ ]:
def generate_anndata_from_sim(sim_res):
    """Generate AnnData object from the simulation results
    
    Time is stored in the time dimension
    """
    
    var_tmp = pd.DataFrame({'gene': pseudo_time_reg_deconv.dataset.selected_genes })
    var_tmp = var_tmp.set_index('gene')
    
    return anndata.AnnData(
        X = sim_res['x_ng'].numpy(),
        var = var_tmp,
        obs = pd.DataFrame({'time': sim_res['t_m']})
    )

## Evaluate Simulation 1 (100 samples; polynomial)

In [ ]:
# Simulate and plot proportions
sim_res = simulate_with_sigmoid_proportions(num_samples=100)
plot_simulated_proportions(sim_res)
matplotlib.pyplot.show()

In [ ]:
simulated_bulk = generate_anndata_from_sim(sim_res)

In [ ]:
ebov_simulated_dataset = DeconvolutionDataset(
    sc_anndata = sc_anndata,
    sc_celltype_col = "Subclustering_reduced",
    bulk_anndata = simulated_bulk,
    bulk_time_col = "time",
    dtype_np = dtype_np,
    dtype = dtype,
    device = device,
    feature_selection_method = 'common' 
)

In [ ]:
pseudo_time_reg_deconv_sim = TimeRegularizedDeconvolution(
    dataset=ebov_simulated_dataset,
    polynomial_degree = 20,
    basis_functions = "polynomial",
    device=device,
    dtype=dtype)

In [ ]:
pseudo_time_reg_deconv_sim.fit_model(n_iters=20_001, verbose=True, log_frequency=1000)

In [ ]:
pseudo_time_reg_deconv_sim.plot_loss()

In [ ]:
pseudo_time_reg_deconv_sim.calculate_composition_trajectories(n_intervals = 1000)

In [ ]:
plot_simulated_proportions(sim_res)

In [ ]:
pseudo_time_reg_deconv_sim.plot_composition_trajectories()

## Polynomial low degree

In [ ]:
pseudo_time_reg_deconv_sim = TimeRegularizedDeconvolution(
    dataset=ebov_simulated_dataset,
    polynomial_degree = 5,
    #basis_functions = "legendre",
    basis_functions = "polynomial",
    device=device,
    dtype=dtype)

In [ ]:
pseudo_time_reg_deconv_sim.fit_model(n_iters=20_001, verbose=True, log_frequency=1000)

In [ ]:
pseudo_time_reg_deconv_sim.plot_loss()

In [ ]:
plot_simulated_proportions(sim_res)

In [ ]:
pseudo_time_reg_deconv_sim.calculate_composition_trajectories(n_intervals = 1000)

In [ ]:
pseudo_time_reg_deconv_sim.plot_composition_trajectories()

### With legendre polynomials

In [ ]:
pseudo_time_reg_deconv_sim = TimeRegularizedDeconvolution(
    dataset=ebov_simulated_dataset,
    polynomial_degree = 3,
    basis_functions = "legendre",
    device=device,
    dtype=dtype)
pseudo_time_reg_deconv_sim.fit_model(n_iters=20_001, verbose=True, log_frequency=1000)
pseudo_time_reg_deconv_sim.plot_loss()

In [ ]:
pseudo_time_reg_deconv_sim.calculate_composition_trajectories(n_intervals = 1000)

In [ ]:
plot_simulated_proportions(sim_res)

In [ ]:
pseudo_time_reg_deconv_sim.plot_composition_trajectories()

In [ ]:
### Many ledenre polynomials

In [ ]:
pseudo_time_reg_deconv_sim = TimeRegularizedDeconvolution(
    dataset=ebov_simulated_dataset,
    polynomial_degree = 15,
    basis_functions = "legendre",
    device=device,
    dtype=dtype)
pseudo_time_reg_deconv_sim.fit_model(n_iters=20_001, verbose=True, log_frequency=1000)
pseudo_time_reg_deconv_sim.plot_loss()

In [ ]:
pseudo_time_reg_deconv_sim.calculate_composition_trajectories(n_intervals = 1000)

In [ ]:
plot_simulated_proportions(sim_res)

In [ ]:
pseudo_time_reg_deconv_sim.plot_composition_trajectories()

# Without betas

In [ ]:
pseudo_time_reg_deconv_sim = TimeRegularizedDeconvolution(
    dataset=ebov_simulated_dataset,
    polynomial_degree = 20,
    #basis_functions = "legendre",
    basis_functions = "polynomial",
    use_betas = False,
    device=device,
    dtype=dtype)

In [ ]:
pseudo_time_reg_deconv_sim.fit_model(n_iters=20_001, verbose=True, log_frequency=1000)
pseudo_time_reg_deconv_sim.plot_loss()

In [ ]:
plot_simulated_proportions(sim_res)

In [ ]:
pseudo_time_reg_deconv_sim.calculate_composition_trajectories(n_intervals = 1000)
pseudo_time_reg_deconv_sim.plot_composition_trajectories()

## Error Evaluation

In [ ]:
prediction_error_L1(sim_res, pseudo_time_reg_deconv_sim, n_intervals = 100)

In [ ]:
# subsample on the t_m (LOO or range)
# training and generalization accuracy 
# generalizatoin accuracy vs time
# discontiuity and sin

In [ ]:
# training and generalization accuracy for different gene and cell distortion